### 가사 수집

In [ ]:
# ==============================================================================
# 1. 필수 라이브러리 설치 및 임포트
# ==============================================================================
!pip install selenium tqdm pandas python-Levenshtein thefuzz
!apt-get update
!apt-get install -y chromium-browser xvfb

import time
import random
import pandas as pd
import os
import re
from urllib.parse import urljoin, quote
from tqdm.auto import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from thefuzz import fuzz

# ==============================================================================
# 2. Colab 환경을 위한 셀레니움 및 기본 설정
# ==============================================================================
INPUT_FILE = 'jpop_chart_list.csv'
OUTPUT_FILE = 'jpop_lyrics_collection.csv'
FAILED_FILE = 'failed_list.txt'
SIMILARITY_THRESHOLD = 65

def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument(f'--user-data-dir=/tmp/chrome_user_data_{random.randint(10000, 99999)}')
    chrome_options.add_argument("--disable-gpu")
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    chrome_options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(30)
    return driver

def normalize_string(text):
    text = re.sub(r'\(.*?\)|\[.*?\]', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s\u3040-\u30ff\u3131-\uD79D\u4e00-\u9fff]', '', text)
    return text.lower().strip()

def clean_search_query(artist, title):
    cleaned_title = re.sub(r'\s*feat\..*$', '', title, flags=re.IGNORECASE)
    cleaned_title = re.sub(r'\s*prod\..*$', '', cleaned_title, flags=re.IGNORECASE)
    cleaned_artist = artist.replace('.', '')
    return cleaned_artist, cleaned_title

# ==============================================================================
# 3. 사이트별 가사 수집 함수 (벅스 전용)
# ==============================================================================

def scrape_from_bugs(driver, artist, title):
    """벅스 '곡' 탭에서 검색하여 가사를 수집하는 함수"""
    search_query = f"{artist} - {title}"
    base_url = "https://music.bugs.co.kr"
    search_url = f"{base_url}/search/track?q={quote(search_query)}"

    try:
        driver.get(search_url)
        wait = WebDriverWait(driver, 10)
        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'trackList')]")))
        except TimeoutException:
            return None

        search_results = driver.find_elements(By.XPATH, "//table[contains(@class, 'trackList')]/tbody/tr")
        if not search_results:
            return None

        for result in search_results:
            try:
                found_title = result.find_element(By.CSS_SELECTOR, 'p.title a').text
                found_artist = result.find_element(By.CSS_SELECTOR, 'p.artist a').text

                title_score = fuzz.partial_ratio(normalize_string(title), normalize_string(found_title))
                artist_score = fuzz.partial_ratio(normalize_string(artist), normalize_string(found_artist))

                if title_score >= SIMILARITY_THRESHOLD and artist_score >= SIMILARITY_THRESHOLD:
                    print(f"  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 {title_score}%, 아티스트 {artist_score}%)")
                    track_info_link_element = result.find_element(By.CSS_SELECTOR, 'a.trackInfo')
                    relative_link = track_info_link_element.get_attribute('href')

                    song_link_full = urljoin(base_url, relative_link)
                    driver.get(song_link_full)

                    wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="lyricsContainer"]//xmp')))
                    lyrics_element = driver.find_element(By.XPATH, '//div[@class="lyricsContainer"]//xmp')
                    lyrics = lyrics_element.get_attribute("innerHTML").strip()
                    print("  🎶 벅스 가사 수집 성공!")

                    return {'source': 'Bugs', 'lyrics': lyrics, 'translation': ''}
            except (NoSuchElementException, TimeoutException):
                continue # 가사가 없으면 다음 검색 결과로

    except Exception as e:
        print(f"  - 벅스 검색/수집 중 예외 발생: {e}")

    return None

# ==============================================================================
# 4. 메인 실행 로직
# ==============================================================================
print("1단계: 데이터 사전 준비 시작...")
if not os.path.exists(INPUT_FILE):
    print(f"❌ 오류: 원본 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 파일을 업로드해주세요.")
else:
    df = pd.read_csv(INPUT_FILE)
    unique_songs_df = df[['artist', 'title']].drop_duplicates().reset_index(drop=True)

    print("\n2단계: 수집 대상 목록 준비 시작...")
    try:
        completed_df = pd.read_csv(OUTPUT_FILE)
        completed_songs = set(zip(completed_df['artist'], completed_df['title']))
        print(f"📄 기존 수집 파일 '{OUTPUT_FILE}'에서 {len(completed_songs)}개의 성공 데이터를 확인했습니다.")
    except (FileNotFoundError, pd.errors.EmptyDataError):
        completed_songs = set()
        pd.DataFrame(columns=['artist', 'title', 'source', 'lyrics', 'translation']).to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    unique_songs_df['key'] = list(zip(unique_songs_df['artist'], unique_songs_df['title']))
    to_scrape_df = unique_songs_df[~unique_songs_df['key'].isin(completed_songs)].drop(columns=['key']).reset_index(drop=True)

    if to_scrape_df.empty:
        print("\n🎉 모든 곡의 가사 수집이 이미 완료되었습니다!")
    else:
        print(f"⏭️ {len(completed_songs)}개는 이미 수집되어 건너뜁니다. 남은 곡: {len(to_scrape_df)}개")
        print(f"\n3단계: 총 {len(to_scrape_df)}곡을 대상으로 수집을 시작합니다... (역순 진행)")

        START_INDEX = 0

        if os.path.exists(FAILED_FILE):
            os.remove(FAILED_FILE)

        # [핵심] 수집 대상을 역순으로 변경
        for index, row in tqdm(to_scrape_df.iloc[::-1].iterrows(), total=len(to_scrape_df), desc="J-POP 가사 수집 중 (역순)"):
            if index < START_INDEX:
                continue

            original_artist, original_title = row['artist'], row['title']
            cleaned_artist, cleaned_title = clean_search_query(original_artist, original_title)

            print(f"\n🔍 수집 시도 ({row.name+1}/{len(unique_songs_df)}): {original_artist} - {original_title}")
            driver, scraped_data = None, None
            try:
                driver = setup_driver()

                scraped_data = scrape_from_bugs(driver, cleaned_artist, cleaned_title)

                if scraped_data:
                    new_row = pd.DataFrame([{'artist': original_artist, 'title': original_title, **scraped_data}])
                    new_row.to_csv(OUTPUT_FILE, mode='a', header=False, index=False, encoding='utf-8-sig')
                else:
                    print(f"  ❌ 수집 실패. 실패 목록에 기록합니다.")
                    with open(FAILED_FILE, "a", encoding="utf-8") as f:
                        f.write(f"{original_artist}\t{original_title}\n")
            except Exception as e:
                print(f"  🚨 루프 내에서 예상치 못한 오류 발생: {e}")
                with open(FAILED_FILE, "a", encoding="utf-8") as f:
                    f.write(f"{original_artist}\t{original_title}\n")
            finally:
                if driver:
                    driver.quit()
                time.sleep(random.uniform(2, 4))

        print("\n🎉 모든 가사 수집 작업이 종료되었습니다!")

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (1,914 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

J-POP 가사 수집 중 (역순):   0%|          | 0/1060 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  🎶 벅스 가사 수집 성공!

🔍 수집 시도 (852/1475): 乃紫 - 初恋キラー
  ❌ 수집 실패. 실패 목록에 기록합니다.

🔍 수집 시도 (851/1475): JO1 - Love seeker
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  🎶 벅스 가사 수집 성공!

🔍 수집 시도 (850/1475): あいみょん - 会いに行くのに
  ❌ 수집 실패. 실패 목록에 기록합니다.

🔍 수집 시도 (849/1475): INI - LOUD
  ➡️ 벅스에서 일치하는 곡 발견! (유사도: 제목 100%, 아티스트 100%)
  - 벅스 검색/수집 중 예외 발생: Message: stale element reference: stale element not found
  (Session info: chrome=139.0.7258.68); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
#0 0x560f3998501a <unknown>
#1 0x560f39424a70 <unknown>
#2 0x560f394380bb <unknown>
#3 0x560f39436e82 <unknown>
#4 0x560f3942bfd9 <unknown>
#5 0x560f3942a20f <unknown>
#6 0x560f3942df58 <unknown>
#7 0x560f3942dfe3 <unknown>
#8 0x560f39476335 <unknown>
#9 0x560f39476b01 <unknown>
#10 0x560f3946a5a6 <un

AttributeError: 'float' object has no attribute 'replace'

### 스포티파이 ID 수집

In [ ]:
# Spotify API를 쉽게 사용하게 해주는 라이브러리 설치
!pip install spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

# 1단계에서 복사한 ID와 Secret을 입력하세요.
CLIENT_ID = 'a92f469c87404d4d9167f8042ff97a19'
CLIENT_SECRET = 'c9dc322c0ea74910b93264c9751b47cd'

In [ ]:
# J-Pop Spotify ID 수집기 - 통합 버전
# Spotify API를 사용해 J-Pop 곡들의 Spotify ID를 자동으로 수집하고 검증하는 도구

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import re
import numpy as np
from difflib import SequenceMatcher
import os
from typing import Optional, List, Dict

class SpotifyIDCollector:
    """J-Pop 곡들의 Spotify ID를 수집하고 검증하는 클래스"""

    def __init__(self, client_id: str, client_secret: str):
        """
        Spotify API 인증 및 초기화

        Args:
            client_id: Spotify API 클라이언트 ID
            client_secret: Spotify API 클라이언트 시크릿
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.sp = None
        self._authenticate()

    def _authenticate(self):
        """Spotify API 인증"""
        try:
            auth_manager = SpotifyClientCredentials(
                client_id=self.client_id,
                client_secret=self.client_secret
            )
            self.sp = spotipy.Spotify(auth_manager=auth_manager)
            print("✅ Spotify API 인증에 성공했습니다.")
        except Exception as e:
            print(f"❌ Spotify API 인증에 실패했습니다: {e}")
            raise

    @staticmethod
    def get_similarity(text1: str, text2: str) -> float:
        """두 문자열의 유사도 계산"""
        return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

    @staticmethod
    def clean_artist_name(artist: str) -> str:
        """아티스트 이름 정제"""
        # 괄호 안 내용 제거 및 전각문자 변환
        artist_cleaned = str(artist).split('(')[0].strip()
        return artist_cleaned.translate(str.maketrans("Ａ-Ｚａ-ｚ０-９", "A-Za-z0-9"))

    @staticmethod
    def generate_title_candidates(original_title: str) -> List[str]:
        """다양한 버전의 검색용 제목 후보 생성"""
        candidates = {original_title}

        # 하이픈(-) 뒷부분 제거
        if ' - ' in original_title:
            candidates.add(original_title.split(' - ')[0].strip())

        # TV/애니메이션 관련 정보 제거
        title_cleaned = re.sub(r'[-–]\s*(TV|애니메이션|드라마|극장판).*', '', original_title, flags=re.I).strip()
        title_cleaned = re.sub(r'\s*\*.+', '', title_cleaned).strip()
        title_cleaned = re.sub(r'\s*feat\s*\..*', '', title_cleaned, flags=re.I).strip()
        title_cleaned = re.sub(r'\s*ft\s*\..*', '', title_cleaned, flags=re.I).strip()
        candidates.add(title_cleaned)

        # 괄호 제거
        title_no_parens = re.sub(r"[\(（\[【].*?[\)）\]】]", "", title_cleaned).strip()
        candidates.add(title_no_parens)

        # 괄호 안 내용 추출
        in_parens = re.findall(r'[\(（](.*?)[\)）]', original_title)
        if in_parens:
            candidates.add(in_parens[0].strip())

        # 슬래시(/) 앞 내용
        if '/' in title_no_parens:
            candidates.add(title_no_parens.split('/')[0].strip())

        return list(filter(None, candidates))

    def search_by_keyword(self, artist: str, title: str) -> Optional[str]:
        """1단계: 키워드 기반 빠른 검색"""
        artist_cleaned = self.clean_artist_name(artist)
        title_candidates = self.generate_title_candidates(title)

        for search_title in title_candidates:
            if not search_title:
                continue

            query = f"artist:{artist_cleaned} track:{search_title}"
            try:
                results = self.sp.search(q=query, type='track', limit=1, market='JP')
                time.sleep(0.5)

                if results['tracks']['items']:
                    track_id = results['tracks']['items'][0]['id']
                    print(f"🔍 [1단계 성공] {title} (검색어: '{search_title}') | ID: {track_id}")
                    return track_id

            except Exception as e:
                print(f"🚨 [1단계 오류] {title} 처리 중: {e}")
                return "ERROR"

        return None

    def search_in_discography(self, artist: str, title: str) -> Optional[str]:
        """2단계: 아티스트 디스코그래피 전체 탐색"""
        artist_candidates = [self.clean_artist_name(artist)]

        # 괄호 안 영어 이름도 후보에 추가
        in_parens = re.findall(r'[\(（](.*?)[\)）]', artist)
        if in_parens:
            artist_in_parens = in_parens[0].split('/')[0].strip()
            if all(ord(c) < 128 for c in artist_in_parens):  # 영어/숫자/기호만
                artist_candidates.append(artist_in_parens)

        for artist_name in artist_candidates:
            print(f"   아티스트 '{artist_name}'로 디스코그래피 탐색...")
            track_id = self._search_artist_albums(artist_name, title)
            if track_id:
                return track_id

        return None

    def _search_artist_albums(self, artist_name: str, title_to_find: str) -> Optional[str]:
        """특정 아티스트의 모든 앨범에서 곡 검색"""
        try:
            # 아티스트 검색
            results = self.sp.search(q=f'artist:{artist_name}', type='artist', limit=1)
            if not results['artists']['items']:
                return None

            artist_id = results['artists']['items'][0]['id']

            # 앨범 목록 가져오기
            albums_response = self.sp.artist_albums(
                artist_id,
                album_type='album,single',
                limit=50
            )
            albums = albums_response['items']

            best_match = {'id': None, 'similarity': 0.0, 'title': ''}
            title_versions = self.generate_title_candidates(title_to_find)

            # 각 앨범의 트랙들과 비교
            for album in albums:
                tracks = self.sp.album_tracks(album['id'])['items']
                for track in tracks:
                    for version in title_versions:
                        similarity = self.get_similarity(version, track['name'])
                        if similarity > best_match['similarity']:
                            best_match.update({
                                'id': track['id'],
                                'similarity': similarity,
                                'title': track['name']
                            })
                time.sleep(0.3)

            # 70% 이상 유사도일 때만 채택
            if best_match['similarity'] > 0.70:
                print(f"✨ [2단계 성공] '{title_to_find}' -> '{best_match['title']}' "
                      f"(유사도: {best_match['similarity']:.0%})")
                return best_match['id']

        except Exception as e:
            print(f"🚨 [2단계 오류] {artist_name} 탐색 중: {e}")

        return None

    def get_spotify_id(self, row) -> Optional[str]:
        """하이브리드 방식으로 Spotify ID 검색"""
        artist, title = row['artist'], row['title']

        # 1단계: 키워드 검색
        track_id = self.search_by_keyword(artist, title)

        # 2단계: 디스코그래피 검색 (1단계 실패시)
        if track_id is None:
            print(f"↪️  1단계 실패, 2단계 디스코그래피 탐색 시작: {title}")
            track_id = self.search_in_discography(artist, title)

        if track_id is None or track_id == "ERROR":
            print(f"⚠️ [최종 실패] {artist} - {title}의 트랙을 찾을 수 없습니다.")
            return None

        return track_id

    def verify_spotify_ids(self, df: pd.DataFrame) -> pd.DataFrame:
        """수집된 Spotify ID들을 검증하고 잘못된 것들 제거"""
        print("\n--- Spotify ID 검증 및 정리 시작 ---")

        # 유효한 ID 형식 확인 (22자리 영문/숫자)
        if 'spotify_id' in df.columns:
            valid_ids_mask = df['spotify_id'].str.match(r'^[a-zA-Z0-9]{22}$', na=False)
            invalid_rows = df[~valid_ids_mask & df['spotify_id'].notna()]

            if not invalid_rows.empty:
                print(f"🗑️ {len(invalid_rows)}개의 잘못된 형식 ID를 삭제합니다.")
                df.loc[invalid_rows.index, 'spotify_id'] = np.nan

        # 아티스트 이름 일치 여부 확인
        verify_df = df[df['spotify_id'].str.match(r'^[a-zA-Z0-9]{22}$', na=False)].copy()

        if not verify_df.empty:
            mismatched_indices = []
            track_ids = verify_df['spotify_id'].tolist()

            # 50개씩 배치로 처리
            for i in range(0, len(track_ids), 50):
                batch_ids = track_ids[i:i+50]
                try:
                    spotify_tracks = self.sp.tracks(batch_ids)['tracks']

                    for j, track_info in enumerate(spotify_tracks):
                        if track_info is None:
                            continue

                        original_index = verify_df.index[i + j]
                        original_artist = verify_df.loc[original_index, 'artist']
                        spotify_artists = ', '.join([artist['name'] for artist in track_info['artists']])

                        main_original_artist = self.clean_artist_name(original_artist).lower()
                        if main_original_artist not in spotify_artists.lower():
                            mismatched_indices.append(original_index)

                except Exception as e:
                    print(f"검증 중 오류: {e}")
                    continue

            if mismatched_indices:
                print(f"🗑️ 아티스트 불일치 ID {len(mismatched_indices)}개를 추가로 삭제합니다.")
                df.loc[mismatched_indices, 'spotify_id'] = np.nan
            else:
                print("✅ 모든 유효 ID가 정확하게 일치합니다.")

        print("--- ID 검증 완료 ---\n")
        return df

    def process_dataset(self, input_file: str, output_file: str = None) -> pd.DataFrame:
        """전체 데이터셋 처리 메인 함수"""
        # 데이터 로드
        try:
            df = pd.read_csv(input_file)
            print(f"📄 '{input_file}' 파일을 성공적으로 불러왔습니다.")

            if 'spotify_id' not in df.columns:
                df['spotify_id'] = np.nan

        except FileNotFoundError:
            print(f"❌ 파일 '{input_file}'을 찾을 수 없습니다.")
            return None

        # 기존 ID 검증 및 정리
        df = self.verify_spotify_ids(df)

        # 누락된 ID 수집
        target_df = df[df['spotify_id'].isnull()].copy()

        if target_df.empty:
            print("🎉 모든 곡의 Spotify ID가 이미 수집되었습니다!")
        else:
            print(f"🎵 총 {len(df)}곡 중 ID가 없는 {len(target_df)}곡에 대한 검색을 시작합니다...")

            # ID 수집 진행
            target_df['spotify_id'] = target_df.apply(self.get_spotify_id, axis=1)

            # 원본 데이터프레임에 업데이트
            df.update(target_df)

            print("\n✨ ID 수집 작업이 완료되었습니다.")

        # 최종 정리: ID가 없는 행 제거
        initial_count = len(df)
        df = df.dropna(subset=['spotify_id']).copy()
        final_count = len(df)

        removed_count = initial_count - final_count
        if removed_count > 0:
            print(f"🗑️ ID를 찾을 수 없는 {removed_count}개의 곡을 최종 데이터에서 제외했습니다.")

        print(f"📊 최종 결과: {final_count}개의 곡이 유효한 Spotify ID와 함께 저장되었습니다.")

        # 결과 저장
        if output_file is None:
            output_file = input_file.replace('.csv', '_with_spotify_ids.csv')

        df.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"✅ 최종 결과가 '{output_file}' 파일로 저장되었습니다.")

        return df

    def show_summary(self, df: pd.DataFrame):
        """작업 결과 요약 출력"""
        print("\n" + "="*50)
        print("📈 작업 완료 요약")
        print("="*50)
        print(f"총 곡 수: {len(df):,}개")
        print(f"Spotify ID 보유: {df['spotify_id'].notna().sum():,}개")
        print(f"성공률: {df['spotify_id'].notna().sum() / len(df) * 100:.1f}%")

        # 상위 아티스트 통계
        if len(df) > 0:
            top_artists = df['artist'].value_counts().head(5)
            print(f"\n🎤 상위 아티스트:")
            for artist, count in top_artists.items():
                print(f"  - {artist}: {count}곡")


def main():
    """메인 실행 함수"""
    # ⚠️ 여기에 본인의 Spotify API 정보를 입력하세요
    CLIENT_ID = 'YOUR_CLIENT_ID'  # 실제 ID로 교체 필요
    CLIENT_SECRET = 'YOUR_CLIENT_SECRET'  # 실제 시크릿으로 교체 필요

    # 입력/출력 파일 경로
    INPUT_FILE = 'jpop_lyrics_collection_filtered.csv'  # 입력 파일명
    OUTPUT_FILE = 'jpop_final_with_spotify_ids.csv'     # 출력 파일명

    try:
        # Spotify ID 수집기 초기화
        collector = SpotifyIDCollector(CLIENT_ID, CLIENT_SECRET)

        # 전체 프로세스 실행
        result_df = collector.process_dataset(INPUT_FILE, OUTPUT_FILE)

        if result_df is not None:
            # 결과 요약 출력
            collector.show_summary(result_df)

            print(f"\n🎊 모든 작업이 성공적으로 완료되었습니다!")
            print(f"결과 파일: {OUTPUT_FILE}")

    except Exception as e:
        print(f"❌ 작업 중 오류가 발생했습니다: {e}")


# 사용 예시
if __name__ == "__main__":
    # 직접 실행 시
    main()

    # 또는 개별적으로 사용하고 싶다면:
    # collector = SpotifyIDCollector('your_id', 'your_secret')
    # df = collector.process_dataset('input.csv', 'output.csv')

✅ Spotify API 인증에 성공했습니다.
✅ 'jpop_lyrics_collection_filtered.csv' 파일을 성공적으로 불러왔습니다.

🎵 Spotify 트랙 ID 검색을 시작합니다...
🔍 [성공] Kenshi Yonezu(켄시 요네즈/米津 玄師) - Lemon  |  ID: 04TshWXkhV1qkqHzf31Hn6
🔍 [성공] NiziU (니쥬) - Make you happy  |  ID: 3sFJmcdXoJ1G2rRW1Hc2x1
🔍 [성공] Lisa Ono(리사 오노/小野リサ) - I Wish You Love  |  ID: 2BJq3tZ0WKGtDtOCo0Dlzy
🔍 [성공] RADWIMPS(래드윔프스) - なんでもないや (movie ver.) / Nandemonaiya (movie ver.)  |  ID: 7dEfa89dZfo6CQPdsgGCF6
⚠️ [실패] RADWIMPS(래드윔프스) - 前前前世 (movie ver.) / Zenzenzense (movie ver.)의 트랙을 찾을 수 없습니다.
🔍 [성공] aimyon - Kimi Wa Rock Wo Kikanai  |  ID: 59eluCMn0XbOWqeWQ91FTM
🔍 [성공] OFFICIAL HIGE DANDISM - Pretender  |  ID: 15HNdxGKNCIO9pgaY4n7FU
⚠️ [실패] RADWIMPS(래드윔프스) - 夢灯籠 / Dream lantern의 트랙을 찾을 수 없습니다.
🔍 [성공] Gen Hoshino(호시노 겐/星野 源) - 恋 / Koi  |  ID: 1vNwRQj10QOLTOTyQYvD0s
🔍 [성공] aimyon - Marigold  |  ID: 5NqGfELjcdvRIUuhgZJ34W
⚠️ [실패] RADWIMPS(래드윔프스) - 愛にできることはまだあるかい / Is There Still Anything That Love Can Do? (Movie Edit)의 트랙을 찾을 수 없습니다.
⚠️ [실패] RADWIMPS(래드윔프스) - グランド

### 한글(일어발음, 해석) 데이터 정제

In [ ]:
pip install pandas langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=cc9413442b1e10ef0d80d242f31b3f260af06b06fa8e408dd5de0daff375d6a4
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [2]:
import re
import pandas as pd
from tqdm import tqdm

# --- 진행률 표시 설정 ---
tqdm.pandas(desc="Lyrics Cleaning Progress")

# --- 한글 제거 기반 일본어/영어 가사 정제 함수 ---
def clean_jpop_lyrics_simple(raw_lyrics):
    """
    일본어 가사 정제 함수 (한글 포함 줄 제거)
    """
    if not isinstance(raw_lyrics, str):
        return ""

    lines = [line.strip() for line in raw_lyrics.split('\n')]
    cleaned_lines = []

    for line in lines:
        if not line:
            continue

        # 한글 포함 줄 제거
        if re.search(r'[\uAC00-\uD7AF]', line):
            continue

        # 일본어/영어/기타 문자만 남기기
        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

# --- DataFrame 불러오기 ---
df_main = pd.read_csv("/content/merged_dataset_utf8.csv")

# --- 정제 함수 적용 ---
df_main['lyrics_cleaned'] = df_main['lyrics'].progress_apply(clean_jpop_lyrics_simple)

# --- CSV로 저장 ---
df_main.to_csv(
    "/content/lyrics_cleaned_no_korean.csv",
    index=False,
    encoding='utf-8-sig'
)

print("한글 제거 후 lyrics_cleaned CSV 생성 완료: /content/lyrics_cleaned_no_korean.csv")


Lyrics Cleaning Progress: 100%|██████████| 461/461 [00:00<00:00, 12920.90it/s]

한글 제거 후 lyrics_cleaned CSV 생성 완료: /content/lyrics_cleaned_no_korean.csv


### 중복 제거 로직

In [ ]:
import pandas as pd
from rapidfuzz import fuzz
from tqdm import tqdm
from itertools import combinations

# tqdm pandas 적용
tqdm.pandas(desc="Progress")

# 1️⃣ 데이터 로드
df_main = pd.read_csv("/content/lyrics_cleaned.csv")

# 2️⃣ 정규화 컬럼 생성 (소문자, 공백 제거)
df_main["title_norm"] = df_main["title"].str.lower().str.strip()
df_main["artist_norm"] = df_main["artist"].str.lower().str.strip()

# 3️⃣ merge_reason 초기화
df_main["merge_reason"] = None

# 4️⃣ Spotify ID 기준 단일화
n_before = len(df_main)
canonical_df = df_main.drop_duplicates(subset=["spotify_id"]).copy()
canonical_df["merge_reason"] = "spotify_id"
n_after_id = len(canonical_df)
print(f"[Spotify ID 단일화] {n_before} → {n_after_id} rows")

# 5️⃣ variants 집계
variants = df_main.groupby("spotify_id").agg({
    "title": lambda x: list(set(x)),
    "lyrics_cleaned": lambda x: list(set(x))
}).reset_index()

# merge 후 컬럼 충돌 방지
canonical_df = canonical_df.merge(variants, on="spotify_id", how="left", suffixes=("_orig", "_variants"))

# 컬럼명 정리
canonical_df = canonical_df.rename(columns={
    "title_orig": "title",
    "lyrics_cleaned_orig": "lyrics_cleaned",
    "title_variants": "title_variants",
    "lyrics_cleaned_variants": "lyrics_variants"
})

# 6️⃣ title+artist 동일하지만 ID 다른 후보 자동 병합
title_artist_groups = df_main.groupby(["artist_norm", "title_norm"])
for (artist, title), group in title_artist_groups:
    unique_ids = group["spotify_id"].unique()
    if len(unique_ids) > 1:
        # 대표 ID 선택 (첫번째)
        main_id = unique_ids[0]
        other_ids = unique_ids[1:]
        for oid in other_ids:
            canonical_df.loc[canonical_df["spotify_id"] == main_id, "title_variants"] += canonical_df.loc[canonical_df["spotify_id"] == oid, "title_variants"].values[0]
            canonical_df.loc[canonical_df["spotify_id"] == main_id, "lyrics_variants"] += canonical_df.loc[canonical_df["spotify_id"] == oid, "lyrics_variants"].values[0]
            canonical_df.loc[canonical_df["spotify_id"] == main_id, "merge_reason"] = "spotify_id + title_artist_merge"
        # 중복 제거
        canonical_df = canonical_df[~canonical_df["spotify_id"].isin(other_ids)]

# 7️⃣ fuzzy lyrics 기준 유사도 병합 (90% 이상)
from itertools import combinations

drop_indices = set()  # 병합 후 제거할 idx 기록

for idx1, idx2 in tqdm(combinations(canonical_df.index.tolist(), 2),
                       desc="Fuzzy lyrics merge", total=len(canonical_df)*(len(canonical_df)-1)//2):

    if idx1 in drop_indices or idx2 in drop_indices:
        continue  # 이미 제거될 대상이면 skip

    lyrics1 = str(canonical_df.at[idx1, "lyrics_cleaned"])
    lyrics2 = str(canonical_df.at[idx2, "lyrics_cleaned"])

    if fuzz.ratio(lyrics1, lyrics2) > 90:
        title1 = canonical_df.at[idx1, "title"]
        title2 = canonical_df.at[idx2, "title"]

        print(f"\n🎵 가사 90% 이상 유사 발견:")
        print(f"1: {title1}  |  2: {title2}")
        choice = input("두 곡을 합치시겠습니까? 1(y)/2(n) : ").strip().lower()

        if choice in ['1','y']:
            canonical_df.at[idx1, "title_variants"].extend(canonical_df.at[idx2, "title_variants"])
            canonical_df.at[idx1, "lyrics_variants"].extend(canonical_df.at[idx2, "lyrics_variants"])
            canonical_df.at[idx1, "merge_reason"] = (canonical_df.at[idx1, "merge_reason"] or "") + " + fuzzy_lyrics_merge"
            drop_indices.add(idx2)  # 제거 대상 기록
        else:
            print("병합하지 않고 다음으로 넘어갑니다.")

# 반복 종료 후 한 번에 제거
canonical_df = canonical_df.drop(drop_indices)



# 8️⃣ 최종 중복 검수
duplicate_ids = canonical_df["spotify_id"].duplicated().sum()
duplicate_title_artist = canonical_df.duplicated(subset=["artist_norm", "title_norm"]).sum()
print(f"\n최종 검수: spotify_id 중복={duplicate_ids}, title+artist 중복={duplicate_title_artist}")

# 9️⃣ 최종 canonical dataset 확인
display_cols = ["spotify_id", "artist", "title", "title_variants", "lyrics_variants", "merge_reason"]
print("\nCanonical dataset 예시:")
print(canonical_df[display_cols].head())



[Spotify ID 단일화] 508 → 479 rows


Fuzzy lyrics merge:   4%|▍         | 4552/114481 [00:00<00:07, 13964.98it/s]


🎵 가사 90% 이상 유사 발견:
1: 愛にできることはまだあるかい / Is There Still Anything That Love Can Do? (Movie Edit)  |  2: 愛にできることはまだあるかい / Is There Still Anything That Love Can Do?
두 곡을 합치시겠습니까? 1(y)/2(n) : 1


Fuzzy lyrics merge:  12%|█▏        | 13643/114481 [00:03<00:14, 6820.18it/s]


🎵 가사 90% 이상 유사 발견:
1: Neko  |  2: Neko -The First Take Version
두 곡을 합치시겠습니까? 1(y)/2(n) : 1


...


🎵 가사 90% 이상 유사 발견:
1: カナタハルカ / KANATA HALUKA  |  2: カナタハルカ
두 곡을 합치시겠습니까? 1(y)/2(n) : 1

🎵 가사 90% 이상 유사 발견:
1: New Genesis (UTA from ONE PIECE FILM RED)  |  2: New Genesis
두 곡을 합치시겠습니까? 1(y)/2(n) : 1


Fuzzy lyrics merge: 100%|█████████▉| 114235/114481 [01:20<00:00, 521.66it/s] 


🎵 가사 90% 이상 유사 발견:
1: Subtitle  |  2: Subtitle
두 곡을 합치시겠습니까? 1(y)/2(n) : 1


Fuzzy lyrics merge: 100%|██████████| 114481/114481 [01:22<00:00, 1395.22it/s]


최종 검수: spotify_id 중복=0, title+artist 중복=0

Canonical dataset 예시:
               spotify_id                       artist  \
0  04TshWXkhV1qkqHzf31Hn6  Kenshi Yonezu(켄시 요네즈/米津 玄師)   
1  3sFJmcdXoJ1G2rRW1Hc2x1                   NiziU (니쥬)   
2  2BJq3tZ0WKGtDtOCo0Dlzy         Lisa Ono(리사 오노/小野リサ)   
3  7dEfa89dZfo6CQPdsgGCF6              RADWIMPS(래드윔프스)   
4  2DLrgv7HhJanCuD8L9uJLR              RADWIMPS(래드윔프스)   

                                              title  \
0                                             Lemon   
1                                    Make you happy   
2                                   I Wish You Love   
3  なんでもないや (movie ver.) / Nandemonaiya (movie ver.)   
4      前前前世 (movie ver.) / Zenzenzense (movie ver.)   

                                      title_variants  \
0                                            [Lemon]   
1                                   [Make you happy]   
2                                  [I Wish You Love]   
3  [Nandemonaiya (movie ver.),

In [ ]:
# 10️⃣ CSV 저장 (선택)
canonical_df.to_csv("canonical_jpop_dataset.csv", index=False, encoding="utf-8-sig")

### 토큰화 및 임베딩

In [25]:
!pip install sentence-transformers
!pip install faiss-gpu-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you 

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
import numpy as np
from tqdm import tqdm

def main():
    print("=== J-POP 가사 데이터 통합 전처리 시작 ===\n")

    # --- 1. 데이터 로드 ---
    print("1. 데이터를 로드합니다...")
    try:
        # merged_dataset_utf8.csv 파일 로드 (모든 컬럼 포함)
        # df = pd.read_csv('/content/merged_dataset_utf8.csv') ################
        print(f"✓ '/content/merged_dataset_utf8.csv' 로드 완료")
    except FileNotFoundError:
        print("❌ 오류: '/content/merged_dataset_utf8.csv' 파일을 찾을 수 없습니다.")
        return

    # --- 2. 데이터 정제 ---
    print("2. 데이터를 정제합니다...")
    # 필수 컬럼 확인 및 정제
    required_columns = ['lyrics_cleaned', 'artist', 'title']
    for col in required_columns:
        if col not in df.columns:
            print(f"❌ 오류: '{col}' 컬럼이 없습니다.")
            return

    # 결측값 제거 및 데이터 정제
    initial_count = len(df)
    df.dropna(subset=['lyrics_cleaned'], inplace=True)
    df['summary'] = df.get('summary', '').astype(str).fillna('')
    df.reset_index(drop=True, inplace=True)

    print(f"✓ {initial_count}곡 → {len(df)}곡 (결측값 제거 후)")

    # --- 3. 임베딩 모델 로드 ---
    print("3. 임베딩 모델을 로드합니다...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    print("✓ 다국어 임베딩 모델 로드 완료")

    # --- 4. 한 줄 단위 데이터 생성 ---
    print("4. 가사를 한 줄 단위로 분할합니다...")
    all_lines_metadata = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="가사 분할 중"):
        lyrics = row['lyrics_cleaned']
        lines = [line.strip() for line in lyrics.split('\n') if line.strip()]

        for line_text in lines:
            all_lines_metadata.append({
                'original_song_index': idx,
                'artist': row['artist'],
                'title': row['title'],
                'line_text': line_text
            })

    print(f"✓ 총 {len(all_lines_metadata)}개의 가사 라인 생성")

    # --- 5. 임베딩 벡터 생성 ---
    print("5. 임베딩 벡터를 생성합니다...")

    # 5-1. 한 줄 임베딩
    print("  • 가사 라인 임베딩 생성 중...")
    line_texts = [meta['line_text'] for meta in all_lines_metadata]
    line_embeddings = model.encode(
        line_texts,
        show_progress_bar=True,
        normalize_embeddings=True
    ).astype('float32')

    # 5-2. 통가사 임베딩
    print("  • 전체 가사 임베딩 생성 중...")
    song_embeddings = model.encode(
        df['lyrics_cleaned'].tolist(),
        show_progress_bar=True,
        normalize_embeddings=True
    ).astype('float32')

    # 5-3. 요약문 임베딩 (있는 경우에만)
    summary_embeddings = None
    if 'summary' in df.columns and not df['summary'].str.strip().eq('').all():
        print("  • 요약문 임베딩 생성 중...")
        summary_embeddings = model.encode(
            df['summary'].tolist(),
            show_progress_bar=True,
            normalize_embeddings=True
        ).astype('float32')

    # --- 6. 데이터 저장 ---
    print("6. 데이터를 저장합니다...")

    # 6-1. 메타데이터 저장 (JSON)
    print("  • 메타데이터 저장 중...")
    with open('line_metadata.json', 'w', encoding='utf-8') as f:
        json.dump(all_lines_metadata, f, ensure_ascii=False, indent=2)

    # 곡 메타데이터 (spotify_id 포함 여부 확인)
    song_metadata_columns = ['artist', 'title']
    if 'spotify_id' in df.columns:
        song_metadata_columns.append('spotify_id')

    with open('song_metadata.json', 'w', encoding='utf-8') as f:
        json.dump(df[song_metadata_columns].to_dict('records'), f, ensure_ascii=False, indent=2)

    # 6-2. 임베딩 벡터 저장 (NPY)
    print("  • 임베딩 벡터 저장 중...")
    np.save('line_embeddings.npy', line_embeddings)
    np.save('song_embeddings.npy', song_embeddings)

    if summary_embeddings is not None:
        np.save('summary_embeddings.npy', summary_embeddings)

    print("\n=== 서버 호환성 확인 ===")
    required_server_columns = ['spotify_id', 'tags_normalized', 'summary', 'album_cover_url']
    missing_columns = [col for col in required_server_columns if col not in df.columns]

    if missing_columns:
        print(f"⚠️  서버에서 필요하지만 누락된 컬럼: {missing_columns}")
        print("   - 기본값으로 채워졌지만, 완전한 기능을 위해서는 해당 데이터가 필요합니다.")
    else:
        print("✅ 서버에서 필요한 모든 컬럼이 준비되었습니다.")

    print("\n=== 전처리 완료! ===")
    print(f"✓ 처리된 곡 수: {len(df)}곡")
    print(f"✓ 생성된 가사 라인: {len(all_lines_metadata)}개")
    print(f"✓ 가사 라인 임베딩: {line_embeddings.shape}")
    print(f"✓ 전체 가사 임베딩: {song_embeddings.shape}")
    if summary_embeddings is not None:
        print(f"✓ 요약문 임베딩: {summary_embeddings.shape}")

    print("\n생성된 파일:")
    print("• line_metadata.json - 가사 라인 메타데이터")
    print("• song_metadata.json - 곡 메타데이터")
    print("• line_embeddings.npy - 가사 라인 임베딩 벡터")
    print("• song_embeddings.npy - 전체 가사 임베딩 벡터")
    if summary_embeddings is not None:
        print("• summary_embeddings.npy - 요약문 임베딩 벡터")

    print("\n이 파일들을 Hugging Face Space에 업로드하면 됩니다.")

if __name__ == "__main__":
    main()

=== J-POP 가사 데이터 통합 전처리 시작 ===

1. 데이터를 로드합니다...
✓ '/content/merged_dataset_utf8.csv' 로드 완료
2. 데이터를 정제합니다...
✓ 461곡 → 461곡 (결측값 제거 후)
3. 임베딩 모델을 로드합니다...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

  • 전체 가사 임베딩 생성 중...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

  • 요약문 임베딩 생성 중...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

6. 데이터를 저장합니다...
  • 메타데이터 저장 중...
  • 임베딩 벡터 저장 중...

=== 서버 호환성 확인 ===
✅ 서버에서 필요한 모든 컬럼이 준비되었습니다.

=== 전처리 완료! ===
✓ 처리된 곡 수: 461곡
✓ 생성된 가사 라인: 22806개
✓ 가사 라인 임베딩: (22806, 384)
✓ 전체 가사 임베딩: (461, 384)
✓ 요약문 임베딩: (461, 384)

생성된 파일:
• line_metadata.json - 가사 라인 메타데이터
• song_metadata.json - 곡 메타데이터
• line_embeddings.npy - 가사 라인 임베딩 벡터
• song_embeddings.npy - 전체 가사 임베딩 벡터
• summary_embeddings.npy - 요약문 임베딩 벡터

이 파일들을 Hugging Face Space에 업로드하면 됩니다.


### 임의 데이터 추가 (수동)

In [1]:
import pandas as pd

# 기존 데이터 불러오기
file_path = "/content/merged_dataset_utf8.csv"
df = pd.read_csv(file_path)

# 새로 추가할 데이터 (하드코딩)
new_data = {
    "artist": "	Arai Yumi",
    "title": "루즈의 전언 / ル-ジュの傳言 (마녀 배달부 키키 OST)",
    "lyrics": """
あのひとのママに会うために
아노 히토노 마마니 아우 타메니
그 사람의 어머니를 만나기 위해

今ひとり列車に乗ったの
이마 히토리 렛샤니 놋타노
지금 홀로 열차에 탔어

たそがれせまる街並や車の流れ
타소가레 세마루 마치나미야 쿠루마노 나가레
해질녘이 다가오는 거리와 늘어선 자동차를

横目で追い越して
요코메데 오이코시테
곁눈질하며 앞질러가

あのひとはもう気づくころよ
아노 히토와 모- 키즈쿠코로요
그 사람은 이제 곧 알게 될거야

バスルームにルージュの伝言
바스루-무니 루-쥬노 덴공
욕실에 립스틱으로 적어놓은 메세지를

浮気な恋をはやくあきらめないかぎり
우와키나 코이오 하야쿠 아키라메나이 카기리
바람피운 사랑을 빨리 포기하기 않는 한

家には帰らない
우치니와 카에라나이
집으로 돌아가지 않을거야
·
不安な気持ちを残したまま
후안나 키모치오 노코시타마마
불안한 마음을 남겨둔 채

街は Ding-Dong 遠ざかってゆくわ
마치와 Ding-Dong 토-자캇테 유쿠와
거리는 Ding-Dong 멀어져가

明日の朝 ママから電話で
아스노 아사 마마카라 뎅와데
내일 아침 어머니가 전화를 해

しかってもらうわ My Darling!
시캇테 모라우와 My Darling!
혼내줄거야 My Darling!
-
あのひとは あわててるころよ
아노 히토와 아와테테루 코로요
그 사람은 당황해 있을거야

バスルームにルージュの伝言
바스루-무니 루-쥬노 덴공
욕실에 립스틱으로 적어놓은 메세지에

てあたりしだい友達にたずねるかしら
테아타리 시다이 토모다치니 타즈네루카시라
단서를 찾는대로 친구에게 물어보려나?

私の行く先を
와타시노 유쿠사키오
내 행방을 말야
·
不安な気持ちを残したまま
후안나 키모치오 노코시타마마
불안한 마음을 남겨둔 채

街は Ding-Dong 遠ざかってゆくわ
마치와 Ding-Dong 토-자캇테 유쿠와
거리는 Ding-Dong 멀어져가

明日の朝 ママから電話で
아스노 아사 마마카라 뎅와데
내일 아침 어머니가 전화를 해

しかってもらうわ My Darling!
시캇테 모라우와 My Darling!
혼내줄거야 My Darling!

しかってもらうわ My Darling!
시캇테 모라우와 My Darling!
혼내줄거야 My Darling!
""",

    "lyrics_cleaned": """
    あのひとのママに会うために
今ひとり列車に乗ったの

たそがれせまる街並や車の流れ
横目で追い越して

あのひとはもう気づくころよ
バスルームにルージュの伝言

浮気な恋をはやくあきらめないかぎり
家には帰らない

不安な気持ちを残したまま
街は Ding-Dong 遠ざかってゆくわ

明日の朝 ママから電話で
しかってもらうわ My Darling!

あのひとは あわててるころよ
バスルームにルージュの伝言

てあたりしだい友達にたずねるかしら
私の行く先を

不安な気持ちを残したまま
街は Ding-Dong 遠ざかってゆくわ

明日の朝 ママから電話で
しかってもらうわ My Darling!

しかってもらうわ My Darling!
    """,
    "summary": "바람피운 연인에게 이별을 알리고 집을 떠나는 여자의 노래",
    "tags_normalized": "이별, 모험, 갈등",
    "album_cover_url": "https://image.bugsm.co.kr/album/images/200/1152/115298.jpg?version=20250617002805",
    "bugs_track_id": "115298",
    "spotify_id": "1jw992uwXhqJJ0H42ucTRL"
}

# DataFrame 형태로 변환 후 기존 데이터에 추가
df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

# UTF-8로 저장 (덮어쓰기)
df.to_csv(file_path, index=False, encoding="utf-8-sig")

print("데이터 추가 완료!")
print(df.tail(3))  # 마지막 3개 확인


데이터 추가 완료!
          artist                             title  \
458   嵐 (ARASHI)                          カイト／Kite   
459           瑛人                         香水／Kousui   
460  \tArai Yumi  루즈의 전언 / ル-ジュの傳言 (마녀 배달부 키키 OST)   

                                                lyrics  \
458  치이사나코로니미타 타카쿠톤데이쿠카이토\n하나사나이요우 귯토츠요쿠\n니기리시메테이타이...   
459  夜中にいきなりさ　いつ空いてるのってライン\n요나카니 이키나리사 이츠 아이테루놋테 라인...   
460  \nあのひとのママに会うために\n아노 히토노 마마니 아우 타메니\n그 사람의 어머니를...   

                                        lyrics_cleaned  \
458                                                NaN   
459  夜中にいきなりさ　いつ空いてるのってライン\n君とはもう三年くらい会ってないのにどうしたの\...   
460  \n    あのひとのママに会うために\n今ひとり列車に乗ったの\n\nたそがれせまる街並や...   

                                               summary     tags_normalized  \
458  서로를 사랑하며 함께한 추억을 회상하며 희망에 가득한 미래를 상상하는 아름다운 사랑 노래  ['사랑', '추억', '희망']   
459               옛날의 사랑하는 이와의 추억을 회상하며 이별과 상실을 노래한 노래  ['회상', '이별', '추억']   
460                    바람피운 연인에게 이별을 알리고 집을 떠나는 여자의 노래          이별, 모험, 